<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>

# Along Track Altimetry Analysis

---

## Overview

1. Using CNES altimetry data
1. Visualizing data using hvplot
1. Use xhistogram to plot multidimensional data

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Pandas](https://foundations.projectpythia.org/core/pandas/pandas.html) | Helpful | |
| [Using hvplot](https://hvplot.holoviz.org) | Helpful | Matplotlib knowledge also helpful |
| [Dask](https://docs.dask.org/en/stable/) | Helpful | |
| [xhistogram](https://xhistogram.readthedocs.io/en/latest/) | Helpful | |

- **Time to learn**: 15 minutes

## Imports

---

In [ ]:
import fsspec
import xarray as xr
import numpy as np
import hvplot
import hvplot.dask
import hvplot.pandas
import hvplot.xarray
from xhistogram.xarray import histogram

## Load Data

The analysis ready along-track altimetry data were prepared by CNES. They are catalogged in the Pangeo Cloud Data Catalog here: https://catalog.pangeo.io/browse/master/ocean/altimetry/

We will work with Jason 3.

In [1]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
print(list(cat))
ds = cat['j3'].to_dask()
ds

ModuleNotFoundError: No module named 'intake'

Load somed data into memory:

In [ ]:
ds_ll = ds[['latitude', 'longitude', 'sla_filtered']].reset_coords().astype('f4').load()
ds_ll

Convert to pandas dataframe:

In [ ]:
df = ds_ll.to_dataframe()
df

## Visualize with hvplot

In [ ]:
df.hvplot.scatter(x='longitude', y='latitude', datashade=True)

## Bin using xhistogram

In [ ]:
lon_bins = np.arange(0, 361, 2)
lat_bins = np.arange(-70, 71, 2)

# helps with memory management
ds_ll_chunked = ds_ll.chunk({'time': '5MB'})

sla_variance = histogram(ds_ll_chunked.longitude, ds_ll_chunked.latitude,
                         bins=[lon_bins, lat_bins],
                         weights=ds_ll_chunked.sla_filtered.fillna(0.)**2)

norm = histogram(ds_ll_chunked.longitude, ds_ll_chunked.latitude,
                         bins=[lon_bins, lat_bins])


# let's get at least 200 points in a box for it to be unmasked
thresh = 200
sla_variance = sla_variance / norm.where(norm > thresh)
sla_variance

In [ ]:
sla_variance.load()

In [ ]:
sla_variance.plot(x='longitude_bin', figsize=(12, 6), vmax=0.2)

## Summary

---

 then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
CESM_MOM6.ipynb

## Resources and references
 - This notebook is based on the Pangeo physical oceanography gallery example: https://gallery.pangeo.io/repos/pangeo-gallery/physical-oceanography/02_along_track.html